In [1]:
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from PIL import Image
import random
import torch

from torchvision.datasets import ImageFolder
from torchvision import transforms
from torchvision import models as models
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.backends import cudnn
from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn.metrics import accuracy_score
from scipy.stats import sem
from sklearn.metrics import confusion_matrix

from collections import defaultdict

import albumentations as A

In [2]:
dataset_path = 'dataset/'
image_path = dataset_path+'faces/'

In [3]:
# добавляем рандом
def seed_everything(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    #torch.cuda.manual_seed(seed)

random_seed = 1120
seed_everything(random_seed)

In [4]:
# Преобразуем возрастные лейблы в 8 категорий
group_8class_age ={
    '0-2':['(0, 2)','2'],
    '4-6':['(4, 6)'],
    '8-13':['(8, 12)','13'],
    '15-20':['(15, 20)'],
    '25-32':['(25, 32)','(27, 32)','32','29'],
    '38-43':['(38, 43)','35', '36', '42','(38, 42)'],
    '48-53':['(48, 53)','42'],
    '60-':['(60, 100)']
}

# Алтернативное группирование, по 4ем категориям
group_4class_age={
'Личинус (0-6)':    ['(0, 2)', '2', '(4, 6)', '3'],
'Щегол (8-23)': ['(8, 12)', '(15, 20)', '(8, 23)', '23', '22', '13'],
'Бумер (25-32)':  ['(25, 32)', '(27, 32)', '32', '34', '29'],
'Старпёр 35+':  ['(48, 53)', '(60, 100)', '55','56','57', '58', '(38, 42)','(38, 43)', '(38, 48)', '35', '36', '46', '45', '42'],
}

In [5]:
# формируем лейблы на основе категорий
def map_age(group_to_age):
    age_to_group = {}
    for group in group_to_age.keys():
        age = group_to_age[group]
        for aa in age:
            age_to_group[aa] = group
    return age_to_group

# берем группировку из 4ех категорий
age_grouping = group_4class_age
age_to_label = map_age(age_grouping)

In [6]:
#считываем размеченный данные
test_fold = [pd.read_csv(f"{dataset_path}fold_0_data.txt",sep = "\t")]
test_fold.append(pd.read_csv(f"{dataset_path}fold_1_data.txt",sep = "\t"))
test_fold.append(pd.read_csv(f"{dataset_path}fold_2_data.txt",sep = "\t"))
test_fold.append(pd.read_csv(f"{dataset_path}fold_3_data.txt",sep = "\t"))
test_fold.append(pd.read_csv(f"{dataset_path}fold_4_data.txt",sep = "\t"))

all_age_group = pd.concat([test_fold[0],test_fold[1],test_fold[2], test_fold[3],test_fold[4]]).age.value_counts()
all_age_group = list(all_age_group.index)

In [7]:
drop_columns=['user_id', 'original_image', 'face_id', 'x', 'y', 'dx',
       'dy', 'tilt_ang', 'fiducial_yaw_angle', 'fiducial_score']

for fold, df in enumerate(test_fold):
    
    #полный путь до изображения
    df['image_path'] = image_path + df['user_id'] + '/coarse_tilt_aligned_face.' + \
        df['face_id'].astype('str') + '.' + df['original_image']
   
    #удаляем лишние колонки
    df.drop(drop_columns, axis=1, inplace=True)
    
    df.gender = df.gender.astype(str)
    df.age = df.age.astype(str)
    
    #удаляем строки с пустыми значениями
    df.drop(df[df.gender == 'u'].index, inplace=True)
    df.drop(df[df.gender == 'nan'].index, inplace=True)
    df.drop(df[df.age == 'None'].index, inplace=True)
    
    #лейбл возраста -> лейбл возрастной категории
    include_age = list(age_to_label.keys())
    exclude_age = list(set(all_age_group) - set(include_age))
    
    #удаляем если вдруг не попало ни в одну возрастную категорию
    for exc_age in exclude_age:
        df.drop(df.loc[df['age']==exc_age].index, inplace=True)
        
    #упорядочиваем индексы по возрасту
    df['age'] = df['age'].apply(lambda x: age_to_label[x])
    
    print(f'Fold {fold+1}  : {df.shape}\n')

Fold 1  : (3995, 3)

Fold 2  : (3597, 3)

Fold 3  : (3124, 3)

Fold 4  : (3291, 3)

Fold 5  : (3445, 3)



In [8]:
# 5 тренировочных фолдов, в виде разной последовтальности тестовых? кажется над переделать
train_fold = [pd.concat([test_fold[1],test_fold[2],test_fold[3],test_fold[4]],ignore_index=True)]
train_fold.append(pd.concat([test_fold[0],test_fold[2],test_fold[3],test_fold[4]],ignore_index=True))
train_fold.append(pd.concat([test_fold[0],test_fold[1],test_fold[3],test_fold[4]],ignore_index=True))
train_fold.append(pd.concat([test_fold[0],test_fold[1],test_fold[2],test_fold[4]],ignore_index=True))
train_fold.append(pd.concat([test_fold[0],test_fold[1],test_fold[2],test_fold[3]],ignore_index=True))

# датафрейм со всем фолдами
all_fold = pd.concat([test_fold[0],test_fold[1],test_fold[2],test_fold[3],test_fold[4]],ignore_index=True)

Fold 1  : (13457, 3)
Fold 2  : (13855, 3)
Fold 3  : (14328, 3)
Fold 4  : (14161, 3)
Fold 5  : (14007, 3)


In [9]:
# приводим значения к числовым
gender_to_label_map = {
    'f' : 0,
    'm' : 1
}

# приводим значения к числовым
age_to_label_map = {
    'Личинус (0-6)' :0,
    'Щегол (8-23)' :1,
    'Бумер (25-32)' :2,
    'Старпёр 35+' :3
}

label_to_age_map = {value: key for key, value in age_to_label_map.items()}
label_to_gender_map = {value: key for key, value in gender_to_label_map.items()}

all_fold['age'].replace(age_to_label_map, inplace=True)
all_fold['gender'].replace(gender_to_label_map, inplace=True)

for i, fold in enumerate(train_fold):
    fold['age'].replace(age_to_label_map, inplace=True)
    fold['gender'].replace(gender_to_label_map, inplace=True)

for i, fold in enumerate(test_fold):
    fold['age'].replace(age_to_label_map, inplace=True)
    fold['gender'].replace(gender_to_label_map, inplace=True)

In [12]:
###
### Классы моделей + аугментация (рамки)
###

In [13]:
trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

In [14]:
class AgeDataset(Dataset):
    def __init__(self, path, image_files, labels_age, labels_gender, p_augment=0.5,  validation=False):
        self.path = path
        self.X = image_files
        self.y_age = labels_age
        self.y_gender = labels_gender
        self.resize = A.Resize(160, 160, always_apply=True)
        self.transform = trans
        
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        image = Image.open(self.path + self.X[i])
        image = np.asarray(image)
        image = self.resize(image=image)['image']
        image = self.transform(image)
        label_age = self.y_age[i]
        label_gender = self.y_gender[i]
        
        return torch.tensor(image, dtype=torch.float), torch.tensor(label_age, dtype=torch.long)

In [15]:
class GenderDataset(Dataset):
    def __init__(self, path, image_files, labels_age, labels_gender, p_augment=0.5,  validation=False):
        self.path = path
        self.X = image_files
        self.y_age = labels_age
        self.y_gender = labels_gender
        self.resize = A.Resize(160, 160, always_apply=True)
        self.transform = trans
        
    def __len__(self):
        return (len(self.X))
    
    def __getitem__(self, i):
        image = Image.open(self.path + self.X[i])
        image = np.asarray(image)
        image = self.resize(image=image)['image']
        image = self.transform(image)
        label_age = self.y_age[i]
        label_gender = self.y_gender[i]
        
        return torch.tensor(image, dtype=torch.float), torch.tensor(label_gender, dtype=torch.long)

In [16]:
train_split={}
val_split={}

for fold in range(5):
    #create train-validation stratified split
    sss = StratifiedShuffleSplit(n_splits=10, random_state=random_seed)
    
    train_data = train_fold[fold]['image_path'].copy().reset_index(drop=True).to_list()
    train_gender_label = train_fold[fold]['gender'].copy().reset_index(drop=True).to_list()
    train_age_label = train_fold[fold]['age'].copy().reset_index(drop=True).to_list()
    train_idx, val_idx = list(sss.split(train_data, train_age_label))[0]
    
    print(f'Training data: {len(train_idx)}')
    print(f'Val. data: {len(val_idx)}')

Training data: 12111
Val. data: 1346
Training data: 12469
Val. data: 1386
Training data: 12895
Val. data: 1433
Training data: 12744
Val. data: 1417
Training data: 12606
Val. data: 1401


In [17]:
###
### Гиперпараметры (!!! говнокод. Переделать!!!)
###

In [18]:
print_sample=False
kfold = 1
batchsize = 64
lr_age = 3e-5
lr_gender= 2e-5
num_epochs = 10
p_augment = 0.0
augment=False


#General parameters
to_predict = 'gender'
device = torch.device("cpu")
num_age_classes = train_fold[0].age.value_counts().shape[0]
num_gender_classes = train_fold[0].gender.value_counts().shape[0]
num_age_classes, num_gender_classes

(4, 2)

In [21]:
###
### Обучение
###

In [22]:
#--- Классифиуируем только возраст  ----

In [24]:
if to_predict == 'gender':
    all_accuracy_gender = []
    all_val_loss_gender = []
    all_stat_fold = []
    
    for fold in range(kfold):
        all_stat = defaultdict(list)
        
        # image paths
        train_data = train_fold[fold]['image_path'].copy().reset_index(drop=True).to_list()
        test_data  = test_fold[fold]['image_path'].copy().reset_index(drop=True).to_list()
    
        #get label
        train_age_label = train_fold[fold]['age'].copy().reset_index(drop=True).to_list()
        train_gender_label = train_fold[fold]['gender'].copy().reset_index(drop=True).to_list()
        test_age_label = test_fold[fold]['age'].copy().reset_index(drop=True).to_list()
        test_gender_label = test_fold[fold]['gender'].copy().reset_index(drop=True).to_list()
    
        #create train-validation stratified split
        sss = StratifiedShuffleSplit(n_splits=10, random_state=random_seed)
    
        #split based on age, more balanced for both age and gender
        train_idx, val_idx = list(sss.split(train_data, train_gender_label))[0]
    
        train_idx = list(train_idx)
        val_idx = list(val_idx)
    
        #create dataloader for gender
        train_dataset = GenderDataset('', 
                                          list(np.array(train_data)[train_idx]), 
                                          list(np.array(train_age_label)[train_idx]),
                                          list(np.array(train_gender_label)[train_idx]),
                                          p_augment = p_augment)
        val_dataset   = GenderDataset('', 
                                          list(np.array(train_data)[val_idx]), 
                                          list(np.array(train_age_label)[val_idx]),
                                          list(np.array(train_gender_label)[val_idx]),
                                          validation=True)
        test_dataset = GenderDataset('', 
                                  test_data, 
                                  test_age_label,
                                  test_gender_label, 
                                  validation=True)
    
    
        train_loader = DataLoader(train_dataset, batch_size=batchsize, shuffle=True)
        val_loader   = DataLoader(val_dataset, batch_size=batchsize, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batchsize, shuffle=False)
    
        val_gender_label = list(np.array(train_gender_label)[val_idx])
        val_age_label = list(np.array(train_age_label)[val_idx])
    
    
        model = InceptionResnetV1(
                        classify=True,
                        pretrained='vggface2',
                        num_classes=num_gender_classes)
        model = model.to(device)
    
        #optimizer
        optimizer = optim.AdamW(model.parameters(), lr = lr_gender)
        scheduler = optim.lr_scheduler.MultiStepLR(optimizer, [5,10])
    
        #loss
        criterion = nn.CrossEntropyLoss()
                    
        best_acc_gender = 0
        best_val_loss_gender = 999
        print(f'Fold {fold+1}\n')
        for epoch in range(num_epochs):
            print(f'epoch: {epoch}\n')
            train_loss_gender = 0
            val_loss_gender = 0
        
            #Training
            model.train()
            iterat = 0
            vsego = len(train_loader)
            for batch in train_loader:
    
                print(f'batch_num: {100*(iterat/vsego)}%\n')
                # Load image batch
                batch_data, batch_gender_label = batch
                batch_data = batch_data.to(device)
                batch_gender_label = batch_gender_label.to(device)
                
                iterat = iterat + 1
                # Clear gradients
                optimizer.zero_grad()
            
                with torch.set_grad_enabled(True):
                    
                    pred_gender = model(batch_data)
                    loss_gender = criterion(pred_gender, batch_gender_label)
            
                    train_loss_gender += loss_gender.detach().item()
                    loss_gender.backward()
                    optimizer.step()
            
            #Validation
            model.eval()
            all_pred_gender = torch.empty(0).to(device)
            for batch in val_loader:
            
                # Load image batch
                batch_data, batch_gender_label = batch
                batch_data = batch_data.to(device)
                batch_gender_label = batch_gender_label.to(device)
                
                with torch.set_grad_enabled(False):
                
                    pred_gender = model(batch_data)
                   
                    loss_gender = criterion(pred_gender, batch_gender_label)
            
                    val_loss_gender += loss_gender.detach().item()
                
                    all_pred_gender = torch.cat((all_pred_gender, 
                            nn.functional.softmax(pred_gender.detach(),dim=1)), 0)
                
        
            train_loss_gender /= len(train_loader)
            val_loss_gender /= len(val_loader)
        
            all_pred_gender = all_pred_gender.cpu().numpy()
            pred_label_gender = list(np.argmax(all_pred_gender,axis=1))
       
        
            acc_gender = accuracy_score(val_gender_label, pred_label_gender)
 
            if val_loss_gender < best_val_loss_gender:
                best_acc_gender=acc_gender
                best_val_loss_gender=val_loss_gender
                torch.save(model.state_dict(), f'model{fold}.pth')
            
            all_stat['train_loss'].append(train_loss_gender)
            all_stat['val_loss'].append(val_loss_gender)
            all_stat['val_acc'].append(acc_gender)
            
            print(f'Epoch {epoch} | train loss: {train_loss_gender} | val loss: {val_loss_gender} | accuracy: {round(acc_gender*100, 2)}%')
            scheduler.step()
        
        #INFERENCE
        with torch.no_grad():
            model.load_state_dict(torch.load(f'model{fold}.pth'))
            model.eval()
            test_pred_gender = torch.empty(0).to(device)
            for batch in test_loader:
            
                # Load image batch
                batch_data, batch_gender_label = batch
                batch_data = batch_data.to(device)
                batch_gender_label = batch_gender_label.to(device)
            
                with torch.set_grad_enabled(False):
                
                    pred_gender = model(batch_data)
               
                    test_pred_gender = torch.cat((test_pred_gender, 
                            nn.functional.softmax(pred_gender.detach(),dim=1)), 0)
                
            test_pred_gender = test_pred_gender.cpu().numpy()
            pred_label_gender = list(np.argmax(test_pred_gender,axis=1))
        
            acc_gender = accuracy_score(test_gender_label, pred_label_gender)
            all_stat['test_acc'].append(acc_gender)
            all_stat['conf'].append(confusion_matrix(test_gender_label, pred_label_gender, labels=list(range(num_gender_classes))))
            all_stat['conf_norm'].append(confusion_matrix(test_gender_label, pred_label_gender,normalize='true', labels=list(range(num_gender_classes))))
            all_stat['test_pred'].append(pred_label_gender)
            all_stat['test_target'].append(test_gender_label)
        all_accuracy_gender.append(acc_gender)
        all_val_loss_gender.append(best_val_loss_gender)
        print(f'TEST ACCURACY: {round(acc_gender*100,2)}% | Val. Accuracy: {round(best_acc_gender*100,2)}% | Val. Loss.: {best_val_loss_gender}\n')
        
        all_stat_fold.append(all_stat)

    all_accuracy_gender = np.array(all_accuracy_gender)
    all_val_loss_gender = np.array(all_val_loss_gender)

    mean_accuracy_gender = round(all_accuracy_gender.mean()*100, 2)

    print(f'\nOverall Accuracy: {mean_accuracy_gender} p/m')

Fold 1

epoch: 0

trained: 0



<ipython-input-15-c5aa7d387409>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image, dtype=torch.float), torch.tensor(label_gender, dtype=torch.long)


batch_num: 0.0%

batch_num: 0.5263157894736842%

batch_num: 1.0526315789473684%

batch_num: 1.5789473684210527%

batch_num: 2.1052631578947367%

batch_num: 2.631578947368421%

batch_num: 3.1578947368421053%

batch_num: 3.684210526315789%

batch_num: 4.2105263157894735%

batch_num: 4.736842105263158%

batch_num: 5.263157894736842%

batch_num: 5.7894736842105265%

batch_num: 6.315789473684211%

batch_num: 6.842105263157896%

batch_num: 7.368421052631578%

batch_num: 7.894736842105263%

batch_num: 8.421052631578947%

batch_num: 8.947368421052632%

batch_num: 9.473684210526317%

batch_num: 10.0%

batch_num: 10.526315789473683%

batch_num: 11.052631578947368%

batch_num: 11.578947368421053%

batch_num: 12.105263157894736%

batch_num: 12.631578947368421%

batch_num: 13.157894736842104%

batch_num: 13.684210526315791%

batch_num: 14.210526315789473%

batch_num: 14.736842105263156%

batch_num: 15.263157894736842%

batch_num: 15.789473684210526%

batch_num: 16.315789473684212%

batch_num: 16.84

<ipython-input-15-c5aa7d387409>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image, dtype=torch.float), torch.tensor(label_gender, dtype=torch.long)


batch_num: 0.0%

batch_num: 0.5263157894736842%

batch_num: 1.0526315789473684%

batch_num: 1.5789473684210527%

batch_num: 2.1052631578947367%

batch_num: 2.631578947368421%

batch_num: 3.1578947368421053%

batch_num: 3.684210526315789%

batch_num: 4.2105263157894735%

batch_num: 4.736842105263158%

batch_num: 5.263157894736842%

batch_num: 5.7894736842105265%

batch_num: 6.315789473684211%

batch_num: 6.842105263157896%

batch_num: 7.368421052631578%

batch_num: 7.894736842105263%

batch_num: 8.421052631578947%

batch_num: 8.947368421052632%

batch_num: 9.473684210526317%

batch_num: 10.0%

batch_num: 10.526315789473683%

batch_num: 11.052631578947368%

batch_num: 11.578947368421053%

batch_num: 12.105263157894736%

batch_num: 12.631578947368421%

batch_num: 13.157894736842104%

batch_num: 13.684210526315791%

batch_num: 14.210526315789473%

batch_num: 14.736842105263156%

batch_num: 15.263157894736842%

batch_num: 15.789473684210526%

batch_num: 16.315789473684212%

batch_num: 16.84

...

batch_num: 0.0%

batch_num: 0.5263157894736842%

batch_num: 1.0526315789473684%

batch_num: 1.5789473684210527%

batch_num: 2.1052631578947367%

batch_num: 2.631578947368421%

batch_num: 3.1578947368421053%

batch_num: 3.684210526315789%

batch_num: 4.2105263157894735%

batch_num: 4.736842105263158%

batch_num: 5.263157894736842%

batch_num: 5.7894736842105265%

batch_num: 6.315789473684211%

batch_num: 6.842105263157896%

batch_num: 7.368421052631578%

batch_num: 7.894736842105263%

batch_num: 8.421052631578947%

batch_num: 8.947368421052632%

batch_num: 9.473684210526317%

batch_num: 10.0%

batch_num: 10.526315789473683%

batch_num: 11.052631578947368%

batch_num: 11.578947368421053%

batch_num: 12.105263157894736%

batch_num: 12.631578947368421%

batch_num: 13.157894736842104%

batch_num: 13.684210526315791%

batch_num: 14.210526315789473%

batch_num: 14.736842105263156%

batch_num: 15.263157894736842%

batch_num: 15.789473684210526%

batch_num: 16.315789473684212%

batch_num: 16.84

<ipython-input-15-c5aa7d387409>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(image, dtype=torch.float), torch.tensor(label_gender, dtype=torch.long)


TEST ACCURACY: 94.72% | Val. Accuracy: 97.4% | Val. Loss.: 0.06985765574923293


Overall Accuracy: 94.72 p/m nan


/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/Cellar/jupyterlab/3.0.12/libexec/lib/python3.9/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


NameError: name 'pickle' is not defined